In [39]:
#################################################################################################################
########################## Import Required Packages and Read in File ############################################
#################################################################################################################
import pandas as pd
import numpy as np
import nltk
import re
import numpy as np
import os
import csv 
from sklearn.cluster import KMeans
print(os.getcwd())



os.chdir("/Users/yeimigonzalez/Desktop/MSA 2021/Text Analytics/Text Analytics Project")


/Users/yeimigonzalez/Desktop/MSA 2021/Text Analytics/Text Analytics Project


In [40]:
wine=pd.read_csv('winemag-data_first150k.csv')
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           150930 non-null  int64  
 1   country      150925 non-null  object 
 2   description  150930 non-null  object 
 3   designation  105195 non-null  object 
 4   points       150930 non-null  int64  
 5   price        137235 non-null  float64
 6   province     150925 non-null  object 
 7   region_1     125870 non-null  object 
 8   region_2     60953 non-null   object 
 9   variety      150930 non-null  object 
 10  winery       150930 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [23]:
#################################################################################################################
######################## Only have to convert columns to a list for step-by-step TF-IDF ######################
#################################################################################################################


In [41]:
#################################################################################################################
################################ Document Clustering and Text Pre-processing ####################################
#################################################################################################################

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(wine['description']))
len(norm_corpus)


150930

In [86]:
#################################################################################################################
####################################### Extract TF-IDF Features #################################################
#################################################################################################################
#taking out words that are not descriptor words, ambiguous words, wine names, and descriptor words that were in 4/5 clusters because they gave no significant interpretation

from sklearn.feature_extraction.text import CountVectorizer

stop_words = stop_words + ['quite',
  'wine',
  'features',
  'fruit',
  'plenty',
  'It',
  'either',
  'hitting',
  'sweet',
  'spot',
  'Good',
  'hold',
  'least',
  'acidity',
  'tannins', 
  'flavors',
  'fruits',
  'notes', 
  'drink', 
  'finish', 
  'shows', 
  'good', 
  'palate', 
  'aromas', 
  'spice', 
  'ripe', 
  'shows', 
  'berry', 
  'nose', 
  'rich', 
  'blend',
  'black',
  'mouth', 
  'offers', 
  'bit', 
  'oak', 
  'red', 
  'dry',  
  'offers', 
  'well', 
  'next',  
  'pinot', 
  'cabernet', 
  'dark', 
  'blackberry', 
  'years',
  'chocolate', 
  'plum', 
  'vanilla', 
  'firm', 
  'like', 
  'full', 
  'character', 
  'structure', 
  'fine', 
  'theres', 
  'easy', 
  'little', 
  'cherry', 
  'soft', 
  'cola',  
  'white', 
  'texture', 
  'currant', 
  'touch',
  'crisp', 
  'fruity', 
  'also', 
  'bright', 
  'clean', 
  'blanc', 
  'sauvignon', 
  'franc', 
  'syrah', 
  'verdot', 
  'petit', 
  'cab', 
  'sauvignon merlot', 
  'petit verdot', 
  'chardonnay', 
  'green', 
  'great', 
  'style', 
  'long', 
  'balanced',  
  'wines', 
  'light', 
  'toast', 
  'mineral', 
  'made', 
  'sangiovese',  
  'malbec',
  'meyer',  
  'meyer lemon', 
  'orange', 
  'stone', 
  'noir', 
  'mouthfeel', 
  'big', 
  'currants', 
  'cherries currants', 
  'fresh', 
  'blackberries currants', 
  'merlot', 
  'peel', 
  'best', 
  'flavor',  
  'one', 
  'slightly',  
  'finishes', 
  'imported',  
  'riesling',
  'hints',
  'yellow', 
  'tannic', 
  'buttered', 
  'spicy', 
  'hint', 
  'note', 
  'nice', 
  'balance', 
  'minerality', 
  'creamy',
  'wood aging', 
  'barrel',  
  'sample', 
  'barrel sample', 
  'give', 
  'alcohol',  
  'almost',  
  'tight', 
  'potential', 
  'weight', 
  'needs', 
  'pair', 
  'new', 
  'vineyard', 
  'core',  
  'edge', 
  'young', 
  'showing', 
  'packed', 
  'along', 
  'aging potential', 
  'time', 
  'layers', 
  'simple', 
  'opens', 
  'yet', 
  'feel',
]

cv = CountVectorizer(ngram_range=(1, 2), min_df=10, max_df=0.8, stop_words=stop_words)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix.shape



(150930, 34172)

In [87]:
#################################################################################################################
####################################### Cluster Wine w/ K-Means #################################################
#################################################################################################################
from sklearn.cluster import KMeans

NUM_CLUSTERS = 3
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=25, n_init=5, random_state=42).fit(cv_matrix)
km



KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=25,
       n_clusters=3, n_init=5, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [88]:
#################################################################################################################
####################################### Determines Observations in Each Cluster #################################
#################################################################################################################
from collections import Counter

Counter(km.labels_)

Counter({0: 129957, 1: 10478, 2: 10495})

In [89]:
wine['kmeans_cluster'] = km.labels_

In [104]:
#################################################################################################################
############ Sorting Clusters by 'points' and Including 'description' and 'variety' for Next Step #####################
#################################################################################################################

wine_clusters = (wine[['price','variety', 'kmeans_cluster', 'points']]
                  .sort_values(by=['kmeans_cluster', 'points'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(20))
wine_clusters = wine_clusters.copy(deep=True)

In [108]:
#################################################################################################################
############ Extracting Key-Features from Description and you can change 'points' Below to  #####################
####### 'variety' or 'description' to See Associated Ratings, Wines, and Descriptions Per Cluster ###############
#################################################################################################################
feature_names = cv.get_feature_names()
topn_features = 50


ordered_centroids = km.cluster_centers_.argsort()[:, ::-1]

# get key features for each cluster
# get movies belonging to each cluster
for cluster_num in range(NUM_CLUSTERS):
    key_features = [feature_names[index] 
                        for index in ordered_centroids[cluster_num, :topn_features]]
    wines = wine_clusters[wine_clusters['kmeans_cluster'] == cluster_num]['variety'].values.tolist()
    print('CLUSTER #'+str(cluster_num+1))
    print('Key Features:', key_features)
    print('Popular Wines:', wines)
    print('-'*80)

CLUSTER #1
Key Features: ['citrus', 'raspberry', 'pepper', 'smooth', 'cherries', 'peach', 'juicy', 'wood', 'herbal', 'tart', 'complex', 'elegant', 'age', 'smoky', 'delicious', 'cassis', 'aging', 'dense', 'still', 'licorice', 'pretty', 'lemon', 'concentrated', 'pineapple', 'solid', 'richness', 'dried', 'vintage', 'pear', 'coffee', 'tastes', 'intense', 'honey', 'earthy', 'much', 'color', 'blackberries', 'fullbodied', 'thats', 'herb', 'silky', 'complexity', 'jammy', 'melon', 'lime', 'tropical', 'wild', 'feels', 'apricot', 'herbs']
Popular Wines: ['Bordeaux-style White Blend', 'Syrah', 'Muscat', 'Merlot', 'Chardonnay', 'Red Blend', 'Merlot', 'Chardonnay', 'Red Blend', 'Chardonnay', 'Syrah', 'Muscat', 'Cabernet Sauvignon', 'Cabernet Sauvignon', 'Pinot Noir', 'Prugnolo Gentile', 'Cabernet Blend', 'Syrah', 'Muscat', 'Merlot']
--------------------------------------------------------------------------------
CLUSTER #2
Key Features: ['tobacco', 'leather', 'pepper', 'dried', 'earth', 'cassis', 'l